<font face="verdana">
#########################################################################################################################<br>

# Homework 5<br>

#########################################################################################################################<br>

1.	Compile a list of static links (permalinks) to individual user movie reviews from one particular website. This will be your working dataset for this assignment, as well as for assignments 7 and 8, which together will make up your semester project.   
a.	It does not matter if you use a crawler or if you manually collect the links, but you will need at least 100 movie review links. Note that, as of this writing, the robots.txt file of IMDB.com allows the crawling of user reviews.
b.	Each link should be to a web page that has only one user review of only one movie, e.g., the user review permalinks on the IMDB site.
c.	Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance, superhero, etc.  
d.	Make sure your collection includes reviews of several movies in your chosen genre and that it includes a mix of negative and positive reviews.  
<br>
<br>
2.	Extract noun phrase (NP) chunks from your reviews using the following procedure:
a.	In Python, use BeautifulSoup to grab the main review text from each link.  
b.	Next run each review text through a tokenizer, and then try to NP-chunk it with a shallow parser. 
c.	You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.
<br>
<br>
3.	Output all the chunks in a single list for each review, and submit that output for this assignment. Also submit a brief written summary of what you did (describe your selection of genre, your source of reviews, how many you collected, and by what means).


In [2]:
# Importing NLTK library and all the books
import nltk
#nltk.download()
import os
#import readline
import numpy as np
import pandas as pd
#import nltk

from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize, sent_tokenize
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
#import tabulate
import urllib.request


In [3]:
import imdb
from bs4 import BeautifulSoup
import requests
import re
import json

my_movies = ["sybil","Shutter Island","split"]
BASE_URL = "https://www.imdb.com"

In [4]:

tbl_imdb  = pd.DataFrame(columns = [
    'Movie_Name',
    'Review_Title',
    'Review_Desc',
    'Review_Rating',
    'Review_url'])

i_index=[]
i_index = 0

In [5]:
Review_Desc_all = ""
for movie_name in my_movies:
    ia = imdb.IMDb()
    results = ia.search_movie(movie_name)
    mv = results[0] #First result
    url = ia.get_imdbURL(mv) #URL for first result
    user_review_url = url+'reviews/'
    response = requests.get(user_review_url)
    soup = BeautifulSoup(response.text, 'lxml')
    try:
        #print(soup)
        reviews = soup.select('a.title')
        #print("reviews ",len(reviews))

        links = [a.attrs.get('href') for a in soup.select('a.title')]
        #print(links)

       # tbl_imdb = []

        # Store each item into dictionary (data), then put those into a list (imdb)
        for index in range(0, len(reviews)):
            # Seperate movie into: 'place', 'title', 'year'
            review_title = reviews[index].get_text()
            review_desc=""
            review_rating=""
            
            #print(r_soup)
            try:
                review_url = BASE_URL+ links[index]
                r_response = requests.get(review_url)
                r_soup = BeautifulSoup(r_response.text, 'lxml')
                
                data = json.loads(r_soup.find('script', type='application/ld+json').text)
                review_desc = data['reviewBody']
                review_rating = data['reviewRating']['ratingValue']
            except:
                pass
            
            raw_data = {'Movie_Name':movie_name,
                    'Review_Title':review_title,
                    'Review_Desc':review_desc,
                    'Review_Rating':review_rating,
                    'Review_url': links[index]}
            df_tbl = pd.DataFrame(raw_data,
            columns = [
                        'Movie_Name',
                        'Review_Title',
                        'Review_Desc',
                        'Review_Rating',
                        'Review_url'], index = [i_index + 1])
            tbl_imdb = tbl_imdb.append(df_tbl)
            Review_Desc_all = review_desc + Review_Desc_all
    except:
        print("Error ")
        


In [6]:
#a:
tbl_imdb['Review_Title'] = tbl_imdb['Review_Title'].str.replace('\n','')  
tbl_imdb.reset_index(drop=True, inplace=True)
tbl_imdb

Movie_Name                                       Review_Title  \
0            sybil   Vivid, unsettling true story given enormous s...   
1            sybil   More than 20 years later, I still remember th...   
2            sybil       Shoe-hooks, white coats and the Big Chair...   
3            sybil                                    Excellent Movie   
4            sybil             A fascinating movie detailing Behavior   
5            sybil     I can't emphasize how underrated this film is!   
6            sybil               A Modern Masterpiece of movie making   
7            sybil                                    Endearing Sybil   
8            sybil   Split personality as a metaphor for what we m...   
9            sybil                    Jessamine Milner, the "Grandma"   
10           sybil         My God ! What did that monster do to you!?   
11           sybil   Sybil DVD was both informative and disappointing   
12           sybil                   Get this MOVIE on DVD, PLEASE!!!   
13           sybil                  Deeply haunting movie.Amazing....   
14           sybil   One of the greatest psychological dramas ever...   
15           sybil                             A Triumph In Every Way   
16           sybil                                        Great Movie   
17           sybil   Very good movie!!!!!!! I would watch again an...   
18           sybil   If you liked this film you must read Flora Rh...   
19           sybil   I count this movie to be in an elite group of...   
20           sybil                                        Still fresh   
21           sybil                                              Sybil   
22           sybil   An excellent movie, but it is too hard to fin...   
23           sybil                            This Movie is Damn Good   
24           sybil                          Definitely worth the buy!   
25  Shutter Island                               Movie of the decade.   
26  Shutter Island                    No one likes to be messed with.   
27  Shutter Island   All it Takes is One Line of Dialogue to Make ...   
28  Shutter Island                  The movie is perfect psycho drama   
29  Shutter Island   Don't be side tracked by the obvious twist......   
..             ...                                                ...   
45  Shutter Island              Scorsese Lays a Very Large Rotten Egg   
46  Shutter Island                               The Good and The Bad   
47  Shutter Island                                   A Horrible Movie   
48  Shutter Island                                 Completely idiotic   
49  Shutter Island                             Boring and pretentious   
50           split                     James McAvoy Carries This Film   
51           split              Great movie! James McAvoy is awesome.   
52           split   It maybe gets a bit too fantastical, but it's...   
53           split                                Great story telling   
54           split                                 Shyamalan Is Back!   
55           split                               Love that he's back!   
56           split                                 they didn't get it   
57           split          I don't understand the hate for this film   
58           split   A great suspense story, with a standout perfo...   
59           split   And I Thought M. Night Shyamalan Was Loosing ...   
60           split              Don't believe the One Star reviews...   
61           split                               Not Perfect But Good   
62           split                                          Rating: A   
63           split   Split not only has Shyamalan back on the sadd...   
64           split        The movie that got me interested into film!   
65           split   Superb Acting by James McAvoy, but don't hold...   
66           split                                    Multiple Scares   
67           split       I always look forward on watching this mo

<font face="verdana">
2.	Extract noun phrase (NP) chunks from your reviews using the following procedure:<br>
<p>a.	In Python, use BeautifulSoup to grab the main review text from each link.<br>  
<p>b.	Next run each review text through a tokenizer, and then try to cr.<br>
<p>c.	You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.

In [7]:
from nltk.corpus import stopwords 
from string import punctuation
customStopWords=set(stopwords.words('english')+list(punctuation))

In [8]:
tbl_imdb['word_token']=""
tbl_imdb['tagged']=""
tbl_imdb['subtree']=""

In [9]:
#NP_grammar = "NP: {<DT>?<JJ>*<NN>}"
NP_grammar = r"""
  NP: {<DT|PP\$>?<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
      {<NNP>+}                # chunk sequences of proper nouns
      {<DT>?<JJ.><NN.*>+}
      {<PRP$><NNP>+<NN>}
      {<DT><NN><VBG>}
      {<CD><NN><TO><CD><NN>}
      {<JJR><IN><CD><NN>}
      {<DT><JJS><VBG><NNS>}
"""
for index, row in tbl_imdb.iterrows():
    print("Movie:",row[0],"\nReview heading:",row[1],"\nRating:",row[3])
    #a: 
    text = row[2].strip()
    if row[2].strip() != '':
        tokens = nltk.word_tokenize(text)
        #print("\ntokens ",tokens)
        tbl_imdb.ix[index,'word_token'] = tokens
        
        tagged = nltk.pos_tag(tokens)
        #print("\n tagged ",tagged)
        tbl_imdb.ix[index,'tagged'] = tagged

        entities = nltk.chunk.ne_chunk(tagged)
        #print("\n entities ",entities)
        #tbl_imdb.ix[index,'entities'] = entities
        
        cp = nltk.RegexpParser(NP_grammar)
        result = cp.parse(entities)
        #print("NP Chunk:")
        NPChunk=""
        for subtree in result.subtrees():
            if subtree.label() == 'NP': 
                print(subtree)
                NPChunk = NPChunk + str(subtree) + ""
        
        tbl_imdb.ix[index,'subtree'] = NPChunk
                
       # tbl_imdb.ix[index,'subtree'] = NPChunk
    print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n",)           

Movie: sybil 
Review heading:  Vivid, unsettling true story given enormous stature by Joanne Woodward and Emmy-winning Sally Field. 
Rating: 10
(NP survive/NN)
(NP long-standing/JJ child/NN)
(NP abuse/NN)
(NP food/NN)
(NP water/NN)
(NP car/NN)
(NP a/DT parent/NN)
(NP works/NN)
(NP yesterday/NN)
(NP paper/NN)
(NP an/DT archbishop/NN)
(NP a/DT progressive/JJ church/NN)
(NP the/DT strangulation/NN)
(NP a/DT 15-year-old/JJ girl/NN)
(NP the/DT opposite/JJ page/NN)
(NP a/DT woman/NN)
(NP boyfriend/NN)
(NP a/DT metal/NN)
(NP pipe/NN)
(NP abuse/NN)
(NP a/DT child/NN)
(NP get/NN)
(NP WHILE/NNP IT/NNP)
(NP HAPPENING/NNP)
(NP Somehow/NNP)
(NP some/DT way/NN)
(NP some/DT sort/NN)
(NP mental/JJ toughness/NN)
(NP defense/NN)
(NP mechanism/NN)
(NP the/DT agony/NN)
(NP fear/NN)
(NP emotional/JJ withdrawal/NN)
(NP the/DT titular/JJ victim/NN)
(NP incessant/JJ child/NN)
(NP abuse/NN)
(NP a/DT woman/NN)
(NP order/NN)
(NP a/DT mother/NN)
(NP indescribable/JJ childhood/NN)
(NP nothing/NN)
(NP speed/NN)
(NP

(NP harrowing/JJ movie/NN)
(NP A/DT small/JJ child/NN)
(NP the/DT grip/NN)
(NP insane/JJ mother/NN)
(NP sadistic/JJ torture/NN)
(NP ineffective/JJ husband/NN)
(NP the/DT other/JJ way/NN)
(NP abuse/NN)
(NP a/DT small/JJ performance/NN)
(NP this/DT movie/NN)
(NP anything/NN)
(NP the/DT film/NN)
(NP the/DT part/NN)
(NP the/DT grandmother/NN)
(NP a/DT victim/NN)
(NP prisoner/NN)
(NP the/DT home/NN)
(NP psychotic/JJ daughter/NN)
(NP The/DT difference/NN)
(NP the/DT extent/NN)
(NP daughter/NN)
(NP insanity/NN)
(NP a/DT prisoner/NN)
(NP own/JJ adult/NN)
(NP child/NN)
(NP home/NN)
(NP abuse/NN)
(NP grandchild/NN)
(NP That/DT kind/NN)
(NP helplessness/NN)
(NP hell/NN)
(NP son-in-law/NN)
(NP the/DT police/NN)
(NP any/DT time/NN)
(NP the/DT house/NN)
(NP anything/NN)
(NP a/DT blind/NN)
(NP eye/NN)
(NP the/DT time/NN)
(NP performance/NN)
(NP affecting/NN)
(NP the/DT film/NN)
(NP the/DT film/NN)
(NP film/NN)
(NP a/DT mystery/NN)
(NP few/JJ TV/NN)
(NP nothing/NN)
(NP a/DT mention/NN)
(NP performance

(NP this/DT movie/NN)
(NP the/DT book/NN)
(NP video/NN)
(NP video/NN)
(NP no/DT one/NN)
(NP no/DT avail/NN)
(NP time/NN)
(NP pressure/NN)
(NP much/JJ rubbish/NN)
(NP cant/NN)
(NP an/DT actual/JJ meaning/NN)
(NP truth/NN)
(NP some/DT understanding/NN)
(NP the/DT life/NN)
(NP way/NN)
(NP Please/NNP)
(NP DVD/NNP)
(NP please/NN)
(NP please/NN)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Movie: sybil 
Review heading:  An excellent movie, but it is too hard to find a copy! 
Rating: 10
(NP this/DT movie/NN)
(NP TV/NN)
(NP a/DT copy/NN)
(NP no/DT success/NN)
(NP Dead/NNP)
(NP Chosen/NNP)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Movie: sybil 
Review heading:  This Movie is Damn Good 
Rating: 9
(NP a/DT true/JJ story/NN)
(NP the/DT writer/NN)
(NP capture/NN)
(NP the/DT harrowing/NN)
(NP precision/NN)
(NP intellect/NN)
(NP the/DT aid/NN)
(NP gore/NN)
(NP Psychological/JJ terror/NN)
(NP effective/JJ tool/NN)
(NP the/DT horrifying/NN)
(NP a

(NP side/NN)
(NP The/DT Last/JJ Temptation/NN)
(NP Bringing/NNP Out/NNP)
(NP Dead/NNP)
(NP The/DT cinematography/NN)
(NP ugly/JJ presentation/NN)
(NP modern/JJ horror/NN)
(NP bright/JJ color/NN)
(NP daytime/NN)
(NP dream/NN)
(NP a/DT distinct/JJ feeling/NN)
(NP terror/NN)
(NP a/DT pretty/JJ face/NN)
(NP a/DT great/JJ story/NN)
(NP boot/NN)
(NP the/DT film/NN)
(NP the/DT source/NN)
(NP material/NN)
(NP the/DT movie/NN)
(NP the/DT film/NN)
(NP television/NN)
(NP the/DT silver/NN)
(NP screen/NN)
(NP the/DT film/NN)
(NP the/DT story/NN)
(NP the/DT know/NN)
(NP paranoia/NN)
(NP phobia/NN)
(NP the/DT camera/NN)
(NP the/DT mental/NN)
(NP the/DT luckiest/JJS authors/NNS)
(NP the/DT planet/NN)
(NP work/NN)
(NP the/DT big/JJ screen/NN)
(NP work/NN)
(NP such/JJ treatment/NN)
(NP the/DT risk/NN)
(NP plot/NN)
(NP the/DT book/NN)
(NP section/NN)
(NP tags/NN)
(NP this/DT story/NN)
(NP any/DT length/NN)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Movie: Shutter Island 
Revi

(NP rating/NN)
(NP this/DT film/NN)
(NP this/DT movie/NN)
(NP first/JJ Scorsese/JJ film/NN)
(NP the/DT genius/NN)
(NP this/DT showing/NN)
(NP brilliance/NN)
(NP a/DT captivating/NN)
(NP storyline/NN)
(NP the/DT release/NN)
(NP a/DT press/NN)
(NP release/NN)
(NP that/DT news/NN)
(NP the/DT story/NN)
(NP A/NNP)
(NP federal/JJ marshall/NN)
(NP the/DT island/NN)
(NP partner/NN)
(NP the/DT disappearance/NN)
(NP a/DT patient/NN)
(NP Each/DT scene/NN)
(NP a/DT turn/NN)
(NP whilst/JJ searching/NN)
(NP Dr./NNP Cawley/NNP)
(NP wife/NN)
(NP the/DT movie/NN)
(NP plot/NN)
(NP controversy/NN)
(NP this/DT film/NN)
(NP the/DT plot/NN)
(NP everything/NN)
(NP fruition/NN)
(NP the/DT movie/NN)
(NP anything/NN)
(NP word/NN)
(NP any/DT Scorsese/JJ movie/NN)
(NP the/DT past/NN)
(NP surrealism/NN)
(NP an/DT intense/JJ thriller/NN)
(NP the/DT mind/NN)
(NP corrode/NN)
(NP this/DT film/NN)
(NP the/DT trailer/NN)
(NP a/DT horror/NN)
(NP film/NN)
(NP a/DT lot/NN)
(NP jumpy/NN)
(NP the/DT same/JJ feeling/NN)
(NP t

(NP A/DT welcome/JJ return/NN)
(NP the/DT director/NN)
(NP finest/JJS performances/NNS)
(NP the/DT film/NN)
(NP level/NN)
(NP studio/NN)
(NP no/DT matter/NN)
(NP range/NN)
(NP talent/NN)
(NP surprising/NN)
(NP another/DT level/NN)
(NP the/DT film/NN)
(NP a/DT great/JJ film/NN)
(NP the/DT theater/NN)
(NP suspense/NN)
(NP this/DT story/NN)
(NP the/DT edge/NN)
(NP seat/NN)
(NP a/DT suspense/JJ mode/NN)
(NP the/DT film/NN)
(NP a/DT typical/JJ kidnapping/NN)
(NP film/NN)
(NP the/DT director/NN)
(NP the/DT film/NN)
(NP the/DT end/NN)
(NP this/DT case/NN)
(NP earlier/JJR films/NNS)
(NP imagination/NN)
(NP work/NN)
(NP great/JJ attention/NN)
(NP a/DT great/JJ film/NN)
(NP an/DT original/JJ suspense/thriller/NN)
(NP go/NN)
(NP a/DT review/NN)
(NP this/DT film/NN)
(NP the/DT mark/NN)
(NP this/DT time/NN)
(NP round/NN)
(NP opinion/NN)
(NP M/NNP Night/NNP)
(NP budget/NN)
(NP work/NN)
(NP storytelling/NN)
(NP McEvoy/NNP)
(NP the/DT risk/NN)
(NP Q/NNP)
(NP A/NNP)
(NP a/DT decent/JJ humble/NN)
(NP ma

In [10]:
tbl_imdb

Movie_Name                                       Review_Title  \
0            sybil   Vivid, unsettling true story given enormous s...   
1            sybil   More than 20 years later, I still remember th...   
2            sybil       Shoe-hooks, white coats and the Big Chair...   
3            sybil                                    Excellent Movie   
4            sybil             A fascinating movie detailing Behavior   
5            sybil     I can't emphasize how underrated this film is!   
6            sybil               A Modern Masterpiece of movie making   
7            sybil                                    Endearing Sybil   
8            sybil   Split personality as a metaphor for what we m...   
9            sybil                    Jessamine Milner, the "Grandma"   
10           sybil         My God ! What did that monster do to you!?   
11           sybil   Sybil DVD was both informative and disappointing   
12           sybil                   Get this MOVIE on DVD, PLEASE!!!   
13           sybil                  Deeply haunting movie.Amazing....   
14           sybil   One of the greatest psychological dramas ever...   
15           sybil                             A Triumph In Every Way   
16           sybil                                        Great Movie   
17           sybil   Very good movie!!!!!!! I would watch again an...   
18           sybil   If you liked this film you must read Flora Rh...   
19           sybil   I count this movie to be in an elite group of...   
20           sybil                                        Still fresh   
21           sybil                                              Sybil   
22           sybil   An excellent movie, but it is too hard to fin...   
23           sybil                            This Movie is Damn Good   
24           sybil                          Definitely worth the buy!   
25  Shutter Island                               Movie of the decade.   
26  Shutter Island                    No one likes to be messed with.   
27  Shutter Island   All it Takes is One Line of Dialogue to Make ...   
28  Shutter Island                  The movie is perfect psycho drama   
29  Shutter Island   Don't be side tracked by the obvious twist......   
..             ...                                                ...   
45  Shutter Island              Scorsese Lays a Very Large Rotten Egg   
46  Shutter Island                               The Good and The Bad   
47  Shutter Island                                   A Horrible Movie   
48  Shutter Island                                 Completely idiotic   
49  Shutter Island                             Boring and pretentious   
50           split                     James McAvoy Carries This Film   
51           split              Great movie! James McAvoy is awesome.   
52           split   It maybe gets a bit too fantastical, but it's...   
53           split                                Great story telling   
54           split                                 Shyamalan Is Back!   
55           split                               Love that he's back!   
56           split                                 they didn't get it   
57           split          I don't understand the hate for this film   
58           split   A great suspense story, with a standout perfo...   
59           split   And I Thought M. Night Shyamalan Was Loosing ...   
60           split              Don't believe the One Star reviews...   
61           split                               Not Perfect But Good   
62           split                                          Rating: A   
63           split   Split not only has Shyamalan back on the sadd...   
64           split        The movie that got me interested into film!   
65           split   Superb Acting by James McAvoy, but don't hold...   
66           split                                    Multiple Scares   
67           split       I always look forward on watching this mo

In [11]:
output = tbl_imdb.groupby('Movie_Name').apply(lambda x: ','.join(x.subtree))

<font face="verdana">
Summary:<br>
    
For this home work i have picked 3 psychological dramas from IMDB:<br>
<br> Sybil: https://www.imdb.com/title/tt0499260/
<br> Split: https://www.imdb.com/title/tt4972582/
<br> Shutter Island: https://www.imdb.com/title/tt1130884/

Web scrapping IMDB User reviews:<br>
1. I used beautiful soup object to get the link for the movie.
2. I used another beautiful soup object to get to user reviews for each movies. The reviews were json object and was parsed to get review description and rating.

Chunking:
<br> 1.For NP chunking, I have defined a regular expression and parsed through each of the entities and collated all "NP" labled entities.
<br> The output for this is the collated NP chunking for 3 movies as below.

In [12]:
output

Movie_Name
Shutter Island    (NP the/DT best/JJS movies/NNS)(NP director/NN...
split             (NP A/DT fantastic/JJ performance/NN)(NP the/D...
sybil             (NP survive/NN)(NP long-standing/JJ child/NN)(...
dtype: object

In [13]:
for i in range(1,3):
    print("\n******************************************************************************************\n")
    print(output[i])


******************************************************************************************

(NP A/DT fantastic/JJ performance/NN)(NP the/DT film/NN)(NP star/NN)(NP reason/NN)(NP this/DT film/NN)(NP Every/DT personality/NN)(NP display/NN)(NP Anya/NNP)(NP breath-taking/NN)(NP a/DT fine/JJ job/NN)(NP a/DT film/NN)(NP M./NNP Night/NNP Shyamalan/NNP)(NP a/DT serious/JJ director/NN)(NP a/DT string/NN)(NP the/DT sequel/NN)(NP the/DT conclusion/NN)(NP series/NN)(NP January/NNP),(NP This/DT movie/NN)(NP the/DT next/JJ character/NN)(NP performance/NN)(NP a/DT man/NN)(NP every/DT part/NN)(NP The/DT end/NN)(NP anything/NN)(NP Well/NNP)(NP today/NN)(NP the/DT mistake/NN)(NP a/DT different/JJ movie/NN)(NP the/DT worst/JJS movies/NNS),(NP this/DT movie/NN)(NP last/JJ year/NN)(NP consideration/NN)(NP the/DT plot/NN)(NP a/DT man/NN)(NP dissociative/JJ identity/NN)(NP disorder/NN)(NP this/DT movie/NN)(NP this/DT movie/NN)(NP a/DT bit/NN)(NP an/DT awesome/JJ thriller/NN)(NP The/DT movie/NN)(NP an/DT awe

<font face="verdana">
#########################################################################################################################<br>

# Homework 7<br>
#########################################################################################################################<br>

Cluster the reviews that you collected in homework 5, by doing the following:
1. In Python, select any one of the clustering methods covered in this course. Run it over the collection of reviews, and show at least two different ways of clustering the reviews, e.g., changing k in k-Means clustering or changing where you “cut” in Agnes or Diana.
2. Try to write a short phrase to characterize (give a natural interpretation of) what each cluster is generally centered on semantically. Is this hard to do in some cases? If so, make note of that fact.
3. Explain which of the two clustering results from question 1 is preferable (if one of them is), and why.
Submit all of your inputs and outputs and your code for this assignment, along with a brief written explanation of your findings.

<font face="verdana">
#########################################################################################################################<br>
<br> 

## NOTE: to adapt to Python 3, I have modified the code from the text book.

#########################################################################################################################<br>


### AS Requested by Brook
    
#########################################################################################################################<br>

In [14]:
import platform; 
print(platform.platform())
import sys; print("Python", sys.version)
import nltk; print( "nltk", nltk.__version__)
#from bs4 import BeautifulSoup, SoupStrainer
import requests; print ("requests", requests.__version__)

try :
    from urllib2 import Request, urlopen
except :
    from urllib.request import Request, urlopen

import re; 
print("re", re.__version__)

#from pattern3.en import parsetree
import os
print (os.environ['CONDA_DEFAULT_ENV'])

Windows-10-10.0.17134-SP0
Python 3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
nltk 3.2.3
requests 2.14.2
re 2.2.1
base


<font face="verdana">
#########################################################################################################################<br>
<br>
<br> 
    
## All required definitions for normalization of corpus

## Code pulled from homework 6

In [33]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = [] 
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [16]:
tbl_imdb.columns

Index(['Movie_Name', 'Review_Title', 'Review_Desc', 'Review_Rating',
       'Review_url', 'word_token', 'tagged', 'subtree'],
      dtype='object')

In [17]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [18]:
movie_titles = ""
synopses = ""
titles = ""

In [19]:
#synopses = tbl_imdb.groupby('Movie_Name').apply(lambda x: ','.join(x.Review_Desc))
movie_titles = tbl_imdb['Movie_Name'].tolist()
synopses = tbl_imdb['Review_Desc'].tolist()
titles= tbl_imdb['Review_Title'].tolist()

In [25]:
norm_movie_synopses = normalize_corpus(synopses,
                                       lemmatize=False)
norm_movie_synopses

['survive much less overcome long standing child abuse newscasts littered unusual horrific stories children imprisoned closets chained beds little food water tiny children dying hot sweltering autos stuffed car trunks parent works yesterdays paper alone archbishop progressive church charged strangulation 15 year old girl sexually assaulted years opposite page woman boyfriend charged beating children metal pipe battered bodies bearing marks years abuse child happening somehow way must build sort mental toughness defense mechanism combat agony fear either tuning systematically shutting going deep states denial emotional withdrawal dorsett sally field unforgettable titular victim incessant child abuse woman dissolved sixteen separate distinct personalities order cope mother inflicted indescribable childhood tortures nothing short amazing especially dissociative scenes morphs lightning speed inner family combative self assertive peggy lou mothering suicidal mary vivacious ambitious vicky f

In [79]:
import pandas as pd

reviews = {'synopses': synopses, 'cluster': clusters}

frame = pd.DataFrame(reviews, index = [clusters] , columns = ['synopses', 'cluster'])

In [21]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [26]:
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
from html.parser import HTMLParser
import unicodedata

stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['mr', 'mrs', 'come', 'go', 'get',
                                 'tell', 'listen', 'one', 'two', 'three',
                                 'four', 'five', 'six', 'seven', 'eight',
                                 'nine', 'zero', 'join', 'find', 'make',
                                 'say', 'ask', 'tell', 'see', 'try', 'back',
                                 'also', 'sybil', 'movies','movie','watched','film']
wnl = WordNetLemmatizer()
html_parser = HTMLParser()

def tokenize_text(text):
    tokens = nltk.word_tokenize(text) 
    tokens = [token.strip() for token in tokens]
    return tokens

def expand_contractions(text, contraction_mapping):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text
    
    
#from pattern3.en import tag
from nltk.corpus import wordnet as wn

# Annotate text tokens with POS tags
def pos_tag_text(text):
    
    def penn_to_wn_tags(pos_tag):
        if pos_tag.startswith('J'):
            return wn.ADJ
        elif pos_tag.startswith('V'):
            return wn.VERB
        elif pos_tag.startswith('N'):
            return wn.NOUN
        elif pos_tag.startswith('R'):
            return wn.ADV
        else:
            return None
    
    tagged_text = nltk.pos_tag(text)
    tagged_lower_text = [(word.lower(), penn_to_wn_tags(pos_tag))
                         for word, pos_tag in
                         tagged_text]
    return tagged_lower_text
    
# lemmatize text based on POS tags    
def lemmatize_text(text):
    
    pos_tagged_text = pos_tag_text(text)
    lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag
                         else word                     
                         for word, pos_tag in pos_tagged_text]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text
    

def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub(' ', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
    
    
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def keep_text_characters(text):
    filtered_tokens = []
    tokens = tokenize_text(text)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def unescape_html(parser, text):
    
    return parser.unescape(text)

def normalize_corpus(corpus, lemmatize=True, 
                     only_text_chars=False,
                     tokenize=False):
    
    normalized_corpus = []    
    for text in corpus:
        text = html_parser.unescape(text)
        text = expand_contractions(text, CONTRACTION_MAP)
        if lemmatize:
            text = lemmatize_text(text)
        else:
            text = text.lower()
        text = remove_special_characters(text)
        text = remove_stopwords(text)
        if only_text_chars:
            text = keep_text_characters(text)
        
        if tokenize:
            text = tokenize_text(text)
            normalized_corpus.append(text)
        else:
            normalized_corpus.append(text)
            
    return normalized_corpus


def parse_document(document):
    document = re.sub('\n', ' ', document)
    if isinstance(document, str):
        document = document
    elif isinstance(document, unicode):
        return unicodedata.normalize('NFKD', document).encode('ascii', 'ignore')
    else:
        raise ValueError('Document is not string or unicode!')
    document = document.strip()
    sentences = nltk.sent_tokenize(document)
    sentences = [sentence.strip() for sentence in sentences]
    return sentences

In [27]:
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
# ... ref : Text Analytics with Python, Sarker, p. 270
# ... -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def build_feature_matrix(documents, feature_type = 'frequency',
                         ngram_range = (1, 1), min_df = 0.0, max_df = 1.0):

    feature_type = feature_type.lower().strip()  
    
    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary = True, min_df = min_df,
                                     max_df = max_df, ngram_range = ngram_range)
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary = False, min_df = min_df,
                                     max_df = max_df, ngram_range = ngram_range)
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer(min_df = min_df, max_df = max_df, 
                                     ngram_range = ngram_range)
    else:
        raise Exception("Wrong feature type entered. Possible values: 'binary', 'frequency', 'tfidf'")

    feature_matrix = vectorizer.fit_transform(documents).astype(float)
    
    return vectorizer, feature_matrix

In [28]:
def print_cluster_data(cluster_data):
    # print cluster details
    for cluster_num, cluster_details in cluster_data.items():
        print( 'Cluster {} details:'.format(cluster_num))
        print( '-'*20)
        print( 'Key features:', cluster_details['key_features'])
        print( 'Movies in this cluster:')
        print( ', '.join(cluster_details['movies']))
        print( '='*40)

<font face="verdana">
#########################################################################################################################<br>
<br>
<br> 

## Homework 8

<br>
<br> 



Perform a vocabulary-based sentiment analysis of the movie reviews you used in homework 5 and homework 7, by doing the following:
1.	In Python, load one of the sentiment vocabularies referenced in the textbook, and run the sentiment analyzer as explained in the corresponding reference. Add words to the sentiment vocabulary, if you think you need to, to better fit your particular text collection.

2.	For each of the clusters you created in homework 7, compute the average, median, high, and low sentiment scores for each cluster. Explain whether you think this reveals anything interesting about the clusters.

3.	For extra credit, analyze sentiment of chunks as follows:
a.	Take the chunks from homework 5, and in Python, run each chunk individually through your sentiment analyzer that you used in question 1. If the chunk registers a nonneutral sentiment, save it in a tabular format (the chunk, the sentiment score).
b.	Now sort the table twice, once to show the highest negative-sentiment-scoring chunks at the top and again to show the highest positive-sentiment-scoring chunks at the top. Examine the upper portions of both sorted lists, to identify any trends, and explain what you see. 
Submit all of your inputs and outputs and your code for this assignment, along with a brief written explanation of your findings. 
#########################################################################################################################<br>
<br>
<br> 
<br>
<br> 
<br>
<br> 
STEPS:

.1. Table is created to capture cluster, cluster words, positive score, negative score <br>
.2. Movie reviews are passed to tdidf vector matrix and Kmeans 3 clusters are created <br>
.3. Each words in the cluster are lemmantized, pos tagged and sentiment scores are obtained and tabulated <br>
.4. Results and observations are in Summary


In [169]:
tbl_cluster  = pd.DataFrame(columns = [
    'Cluster#',
    'top_word_1',
    'top_word_2',
    'top_word_3',
    'top_word_4',
    'top_word_5',
    'pos_score_word_1',
    'neg_score_word_1',
    'pos_score_word_2',
    'neg_score_word_2',
    'pos_score_word_3',
    'neg_score_word_3',
    'pos_score_word_4',
    'neg_score_word_4',
    'pos_score_word_5',
    'neg_score_word_5'])

i_index=[]
i_index = 0

In [168]:
# Annotate text tokens with POS tags
#n - NOUN 
#v - VERB 
#a - ADJECTIVE 
#s - ADJECTIVE SATELLITE 
#r - ADVERB 
def getLemma(word):
    return WordNetLemmatizer().lemmatize(word)

def pos_tag_word(word):
    #lword = WordNetLemmatizer().lemmatize(word) 
    #print(word) #,": ",lword)
    pos_tag = nltk.pos_tag(word)
    print(word, "is an ", pos_tag[0][1])
    if 'NN' in pos_tag[0][1] or 'NNS' in pos_tag[0][1]:
        return 'n' 
    elif pos_tag[0][1] == 'VB' or pos_tag[0][1] == 'VBN': 
        return 'v'
    elif pos_tag[0][1] == 'JJ': #ADJECTIVE
        return 'a'
    elif pos_tag[0][1] == 'RB':
        return 'r'
    else:
        return 'n'

In [102]:
def tokenize_and_lemme(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            #token = WordNetLemmatizer().lemmatize(token)
            filtered_tokens.append(token)
    lemme = [ WordNetLemmatizer().lemmatize(t) for t in filtered_tokens]
    return lemme

In [55]:
#lemme_movie_synopses = tokenize_and_lemme(norm_movie_synopses)
norm_movie_synopses

['survive much less overcome long standing child abuse newscasts littered unusual horrific stories children imprisoned closets chained beds little food water tiny children dying hot sweltering autos stuffed car trunks parent works yesterdays paper alone archbishop progressive church charged strangulation 15 year old girl sexually assaulted years opposite page woman boyfriend charged beating children metal pipe battered bodies bearing marks years abuse child happening somehow way must build sort mental toughness defense mechanism combat agony fear either tuning systematically shutting going deep states denial emotional withdrawal dorsett sally field unforgettable titular victim incessant child abuse woman dissolved sixteen separate distinct personalities order cope mother inflicted indescribable childhood tortures nothing short amazing especially dissociative scenes morphs lightning speed inner family combative self assertive peggy lou mothering suicidal mary vivacious ambitious vicky f

In [170]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from __future__ import print_function
from nltk.corpus import sentiwordnet as swn

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=1.8, 
                                   max_features=200000,
                                   min_df=0.2, 
                                   stop_words='english',
                                    use_idf=True, 
                                   tokenizer=tokenize_and_lemme)

#%time 
tfidf_matrix = tfidf_vectorizer.fit_transform(norm_movie_synopses) #fit the vectorizer to synopses

#print(tfidf_matrix.shape)

num_clusters = 3

km = KMeans(n_clusters=num_clusters)

#%time 
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()


totalvocab_stemmed = []
totalvocab_tokenized = []
for i in synopses:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)
    
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
terms = tfidf_vectorizer.get_feature_names()
print(vocab_frame)    
    
print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    #print("Cluster %d words:" % i, end='')
    print("Cluster %d words:\n" % i)
    print("-----------------------------")
    
    top_word = ""
    for ind in order_centroids[i, :6]:
        #print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
        print(vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0])
        top_word = top_word + "," + vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0]
        
         # print(vocab_frame.ix[terms[2].split(' ')].values.tolist()[0][0])
       # print(vocab_frame.ix[terms[3].split(' ')].values.tolist()[0][0])
       # print(vocab_frame.ix[terms[4].split(' ')].values.tolist()[0][0])
       # print(vocab_frame.ix[terms[5].split(' ')].values.tolist()[0][0])
       # print(vocab_frame.ix[terms[6].split(' ')].values.tolist()[0][0])
    #print("\ntop_words \n",top_word)
    #print(pos_tag_word([top_word.split(',')[1]]))
    word1 = getLemma(top_word.split(',')[1])
    #top_word_1 = top_word.split(',')[1] + '.' + pos_tag_word([])+'.01'
    top_word_1 = word1 + '.' + pos_tag_word([word1])+'.01'
    word2 = getLemma(top_word.split(',')[2])
    top_word_2 = word2 + '.' + pos_tag_word([word2])+'.01'
    
    word3 = getLemma(top_word.split(',')[3])
    top_word_3 = word3 + '.' + pos_tag_word([word3])+'.01'
    
    word4 = getLemma(top_word.split(',')[4])
    top_word_4 = word4  + '.' + pos_tag_word([word4])+'.01'
    
    word5 = getLemma(top_word.split(',')[5])
    top_word_5 = word5  + '.' + pos_tag_word([word5])+'.01'
    
    print(word1)
    try:
        top_word_1_p = (swn.senti_synset(top_word_1)).pos_score()
        top_word_1_n = (swn.senti_synset(top_word_1)).neg_score()
    
    except:
        print("Exception")
        top_word_1_p = 0.0
        top_word_1_n = 0.0
        
    try:
        top_word_2_p = (swn.senti_synset(top_word_2)).pos_score()
        top_word_2_n = (swn.senti_synset(top_word_2)).neg_score()
    
    except:
        top_word_2_p = 0.0
        top_word_2_n = 0.0
      
    try:
        top_word_3_p = (swn.senti_synset(top_word_3)).pos_score()
        top_word_3_n = (swn.senti_synset(top_word_3)).neg_score()
    except:
        top_word_3_p = 0.0
        top_word_3_n = 0.0
        
    try:
        top_word_4_p = (swn.senti_synset(top_word_4)).pos_score()
        top_word_4_n = (swn.senti_synset(top_word_4)).neg_score()
    except:
        top_word_4_p = 0.0
        top_word_4_n = 0.0
    try:
        top_word_5_p = (swn.senti_synset(top_word_5)).pos_score()
        top_word_5_n = (swn.senti_synset(top_word_5)).neg_score()
    except:
        top_word_5_p = 0.0
        top_word_5_n = 0.0  
        
    raw_data = {'Cluster#':i,
                'top_word_1':top_word_1,
                'top_word_2':top_word_2,
                'top_word_3':top_word_3,
                'top_word_4':top_word_4,
                'top_word_5':top_word_5,
                'pos_score_word_1':float(top_word_1_p),
                'neg_score_word_1':float(top_word_1_n),
                'pos_score_word_2':float(top_word_2_p),
                'neg_score_word_2':float(top_word_2_n),
                'pos_score_word_3':float(top_word_3_p),
                'neg_score_word_3':float(top_word_3_n),
                'pos_score_word_4':float(top_word_4_p),
                'neg_score_word_4':float(top_word_4_n),
                'pos_score_word_5':float(top_word_5_p),
                'neg_score_word_5':float(top_word_5_n)               
               }
    df_tbl = pd.DataFrame(raw_data,
            columns = [
                        'Cluster#',
                        'top_word_1',
                        'top_word_2',
                        'top_word_3',
                        'top_word_4',
                        'top_word_5',
                        'pos_score_word_1',
                        'neg_score_word_1',
                        'pos_score_word_2',
                        'neg_score_word_2',
                        'pos_score_word_3',
                        'neg_score_word_3',
                        'pos_score_word_4',
                        'neg_score_word_4',
                        'pos_score_word_5',
                        'neg_score_word_5'], index = [i_index + 1])
    tbl_cluster = tbl_cluster.append(df_tbl)
    print("raw_data ",raw_data)

                    words
how                   how
doe                  does
one                   one
surviv            survive
much                 much
less                 less
overcom          overcome
long-stand  long-standing
child               child
abus                abuse
newscast        newscasts
are                   are
litter           littered
with                 with
the                   the
more                 more
unusu             unusual
horrif           horrific
stori             stories
children         children
imprison       imprisoned
in                     in
closet            closets
or                     or
chain             chained
to                     to
bed                  beds
with                 with
littl              little
more                 more
...                   ...
from                 from
shyamalan       shyamalan
that                 that
will                 will
deliv             deliver
a                       a
jolt        

<font face="verdana">

## Table summary

In [171]:
tbl_cluster.reset_index(drop= True, inplace=True)
tbl_cluster

Cluster#        top_word_1    top_word_2        top_word_3      top_word_4  \
0        0         time.n.01     many.a.01          way.n.01    watched.v.01   
1        1         like.n.01    great.a.01     thriller.n.01      story.n.01   
2        2  personality.n.01  watched.v.01  performance.n.01  character.n.01   

         top_word_5  pos_score_word_1  neg_score_word_1  pos_score_word_2  \
0     thriller.n.01             0.000             0.000               0.0   
1  performance.n.01             0.125             0.000               0.0   
2         time.n.01             0.000             0.125               0.0   

   neg_score_word_2  pos_score_word_3  neg_score_word_3  pos_score_word_4  \
0               0.0             0.000               0.0               0.0   
1               0.0             0.000               0.0               0.0   
2               0.0             0.125               0.0               0.0   

   neg_score_word_4  pos_score_word_5  neg_score_word_5  
0               0.0             0.000               0.0  
1               0.0             0.125               0.0  
2               0.0             0.000               0.0